# seq2seq: 
### I think generating a new sequence that can be a possible ending of a sentence is a little close to what I am looking for. Ideally I woud like to be able to apply this to car share banner generation. And my goal is to be able to write in a way that can end the input sentence. SInce this is sentence completion, I would like to have has many variety in my result set. i know I dont have 120M images like resnet however I do have 0.1% of that ~ 120K

In [46]:
# !pip install spacy
# !python3 -m spacy download en_core_web_sm

In [2]:
import pickle
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras import backend

import numpy as np
import string

import os
os.getcwd()

import re
from colorama import Fore, Back, Style

Using TensorFlow backend.


In [4]:
with open('/home/ec2-user/SageMaker/MSDS696/wine_msds.pkl', 'rb') as f:
    wine_msds = pickle.load(f)

In [5]:
wine_msds['taster_name'].value_counts()

Unknown Taster        24917
Roger Voss            23560
Michael Schachner     14046
Kerin O’Keefe          9697
Paul Gregutt           8868
Virginie Boone         8708
Matt Kettmann          5730
Joe Czerwinski         4766
Sean P. Sullivan       4461
Anna Lee C. Iijima     4017
Jim Gordon             3766
Anne Krebiehl MW       3290
Lauren Buzzeo          1700
Susan Kostrzewa        1022
Mike DeSimone           461
Jeff Jenssen            436
Alexander Peartree      383
Carrie Dykes            129
Fiona Adams              24
Christina Pickard         6
Name: taster_name, dtype: int64

Wont work with entire set. Making source smaller. If I have the chance, I will try with entire corpus because I will have to create a completely different type of machine, working on this at the same tme.

In [6]:
#tasters = ['Roger Voss', 'Michael Schachner'] # still too much memory 228 is not possible for me 
#tasters = ['Jim Gordon'] # this is 10 times less.
#tasters = ['Virginie Boone']  # still 180gb ram 
tasters = ['Jeff Jenssen']

wine_msds_taster = wine_msds[wine_msds.taster_name.isin(tasters)]
wine_msds_taster.shape

(436, 13)

In [7]:
df_rev = wine_msds_taster[['description']].drop_duplicates()

print(df_rev.shape)
df_rev.head(3)

(436, 1)


,description
98828,This blend of Cabernet Sauvignon and Merlot ha...
5961,This Slovenian Chardonnay has aromas of orange...
6968,"This amber colored, orange-style wine has arom..."


In [8]:
# First some regex to get rid of weird words and replace sentence endings.
def normalize_text(text):
    tm1 = re.sub('[0-9]', '', text, flags=re.DOTALL)
    tm2 = re.sub('\x9d', ' ', tm1, flags=re.DOTALL)
    tm3 = re.sub('\xa0', ' ', tm2, flags=re.DOTALL)
    tm4 = re.sub('•', '', tm3, flags=re.DOTALL)  # extra in case need additional
    tm5 = re.sub('\xad', ' ', tm4, flags=re.DOTALL)
    tm6 = re.sub('\u3000', ' ', tm5, flags=re.DOTALL)
    tm7 = re.sub('`', '', tm6, flags=re.DOTALL) # had to add these, for tm9 anything after would not work
    tm8 = re.sub('´', '', tm7, flags=re.DOTALL)
    tm9 = re.sub('<[^>]…+>–©¡—¨¬°º½%$.-•', '', tm8, flags=re.DOTALL)
    return tm9.replace("\n", "")

In case it works: '\\' was giving me so much trouble. one \ excapes the other. since it is looking for aingle one, removing one and trying my chances.idea is if I remove one \ then would remove all.
No it did not, I was not able to remove \\ what gives? - need to check this

In [9]:
# Removing code syntax from text 
df_rev['description_clean']=df_rev['description'].apply(normalize_text)

In [10]:
# I am not able to remove everything using regex, so doing good old fashion
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_", "‘", "’", "“", "”", "…",
              "`","{","|","}","~","–", "©",'¡',"—","¨","¬","°","º","½","%","$",".","-","•"]
for char in spec_chars:
    df_rev['description_clean'] = df_rev['description_clean'].str.replace(char, '')

In [11]:
print(df_rev['description'])
print(Fore.GREEN + Style.DIM + '\nAfter normalizing text-----\n')
print(df_rev['description_clean'])

# Nothing visible below but text had abnormalities that had to be treated

98828     This blend of Cabernet Sauvignon and Merlot ha...
5961      This Slovenian Chardonnay has aromas of orange...
6968      This amber colored, orange-style wine has arom...
6969      This wine is garnet in color, with aromas of c...
15311     Aromas of freshly mowed grass, lemon pith, and...
                                ...                        
70305     This deep straw-colored wine has a nose of man...
100610    This dry Riesling offers rich floral aromas of...
100615    Made with 100% Cabernet Moravia (an indigenous...
100616    On the nose, spice outweighs fruit, with notes...
91173     A mix of red and black fruits pervade on the n...
Name: description, Length: 436, dtype: object

After normalizing text-----

98828     This blend of Cabernet Sauvignon and Merlot ha...
5961      This Slovenian Chardonnay has aromas of orange...
6968      This amber colored orangestyle wine has aromas...
6969      This wine is garnet in color with aromas of ch...
15311     Aromas of fres

### I will keep the corpus as is before I clean it further. A few wine variety names should not hurt the overall intend and again my intention is to be coherent but not homogenous. With that said the special chars almost always muddle the intent. I am not talking about removing the exclamation mark (!), just the weird chars.

In [12]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [13]:
# Trick: A way to check unique vocabulary
wine_review_corpus = " ".join(df_rev['description_clean'].astype(str))
# Convert list of strings to string
check_vocabulary = convert_list_to_string(wine_review_corpus).lower()
vocabulary = sorted(set(check_vocabulary)) # get rid of duplicates
print('Vocabulary size:', len(vocabulary))
print(vocabulary)

Vocabulary size: 36
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'ä', 'è', 'é', 'ñ', 'ú', 'û', 'ü', 'ž']


Okay we are rolling with this ^. 55 as vocabulary size is decent. RNN example showed us it is nice to have this type of variety however in full sentance generation not sure how it is going to work. 

Update: 50 chars now, Instead of using 119K, I am planning to imitate top 2 reviewers, their total number of reviews come out to ~37K. About 1/3 of original.

Update: Still needed 228 memory even at 8bits, so I decided to take 10% of previous run, at 3.7% of total reviews our new robot is now 'Jim Gordon'

In [14]:
df_rev.head()

,description,description_clean
98828,This blend of Cabernet Sauvignon and Merlot ha...,This blend of Cabernet Sauvignon and Merlot ha...
5961,This Slovenian Chardonnay has aromas of orange...,This Slovenian Chardonnay has aromas of orange...
6968,"This amber colored, orange-style wine has arom...",This amber colored orangestyle wine has aromas...
6969,"This wine is garnet in color, with aromas of c...",This wine is garnet in color with aromas of ch...
15311,"Aromas of freshly mowed grass, lemon pith, and...",Aromas of freshly mowed grass lemon pith and l...


### lower case, and change type to a list. 
I have seen examples where upper case exists in the middle of the word which is a simpler mistake and can give away the bot behind the wine review. For this reason lowering the case across.

In [15]:
df_rev['description_clean'] = df_rev['description_clean'].str.lower()  
reviews_corpus = df_rev['description_clean'].tolist()
reviews_corpus[:10]

# I need to do a better job on the special chars. This method is not working like I like, 
# I can add missing ones to another method. - I did, all done!!!

['this blend of cabernet sauvignon and merlot has aromas of coffee black cherry and dark chocolate it is tart in the mouth with flavors of cherry and red raspberry',
 'this slovenian chardonnay has aromas of orange vanilla toasted almonds and citrus blossom the mouth delivers flavors of honeydew melon pear and apple before a crisp acidic finish',
 'this amber colored orangestyle wine has aromas of canned peaches and apricots on the nose while fresh stone fruits seem to dominate the full bodied palate the finish is creamy and lingers on',
 'this wine is garnet in color with aromas of cherry vanilla red plum and cassis flavors of tart cherry red raspberry and blackberry drive the palate and lead to a lingering finish',
 'aromas of freshly mowed grass lemon pith and lemon grass open for mineraldriven flavors of bartlett pear lemon juice and green apple in the midpalate',
 'this slovenian ribolla gialla has aromas of peach and vanilla with flavors of freshly baked peach pie canned apricots

# Preprocess sentences:

### Preprocess sentences so they can be used to train the encoder decoder model. Find the input and the target sequence from original sentences. Similar to what I have been doing. Instead of each word, this time each sentence is divided into two at each char position. This generates one start and one end for each occurence. Just like the /n and /t from RNN run.

In [16]:
### Generate prefix and suffix sentences
prefix_sentences = []
suffix_sentences = []
# Iterate over each character position in each email
for reviews in reviews_corpus: 
    for index in range(len(reviews)): 
        # Find the prefix and suffix        
        prefix = reviews[:index+1]        
        suffix = '\t' + reviews[index+1:] + '\n'
        
        # Add prefix and sufix to the lists        
        prefix_sentences.append(prefix)        
        suffix_sentences.append(suffix)

In [17]:
len(prefix_sentences) # just curious. This worked with 150GB.

89223

In [18]:
#Vocabulary and mappings
# Initialize vocabulary with the start and end token
vocabulary = set(['\t', '\n'])

# Check each char in each email by iterating for each char in each email
for reviews in reviews_corpus:
    for char in reviews:
        if (char not in vocabulary): 
            vocabulary.add(char) 

# Sort the vocabulary
vocabulary = sorted(vocabulary)  
# Tips: initially I had sorted(list(vocabulary)) 
# I end up getting AttributeError: 'list' object has no attribute 'add'
            

# Create char to int and int to char mapping
char_to_idx = dict((char, idx) for idx, char in enumerate(vocabulary))
idx_to_char = dict((idx, char) for idx, char in enumerate(vocabulary))


Vocabulary and the character to integer and integer to character mapping are done. Next is creating input and target vectors.

In [19]:
# Define input and target vectors
# Find the length of the longest prefix and suffix
max_len_prefix_sent = max([len(prefix) for prefix in prefix_sentences])
max_len_suffix_sent = max([len(suffix) for suffix in suffix_sentences])

In [20]:
# Define a 3-D zero vector for the prefix sentences
input_data_prefix = np.zeros((len(prefix_sentences), max_len_prefix_sent, len(vocabulary)), dtype='float32')
# Define a 3-D zero vector for the suffix sentences
input_data_suffix = np.zeros((len(suffix_sentences), max_len_suffix_sent, len(vocabulary)), dtype='float32')
# Define a 3-D zero vector for the target data
target_data = np.zeros((len(suffix_sentences), max_len_suffix_sent, len(vocabulary)), dtype='float32')

# MemoryError: Unable to allocate 4.61 TiB for an array with shape (28063433, 792, 57) and data type float32

# Run 2, added 5.5TiB. Love the cloud! Still not working though. lol. 

# difference between dtype='uint8' and float32 is exactly what it is 4 times! 


# Here I filled the input and target vectors with proper shape, next is filling the vectors with data
# I will iterate over each char of each sentence and convert the char to a one-hot encoded vector


The input and target vectors are now available. Now fill the vectors with data

In [21]:
# Initialize input and target vectors
for i in range(len(prefix_sentences)):
    # Iterate over each character in each prefix
    for k, ch in enumerate(prefix_sentences[i]):
        # Convert the character to a one-hot encoded vector
        input_data_prefix[i, k, char_to_idx[ch]] = 1
        
    # Iterate over each character in each suffix
    for k, ch in enumerate(suffix_sentences[i]):
        # Convert the character to a one-hot encoded vector
        input_data_suffix[i, k, char_to_idx[ch]] = 1

        # Target data is one timestep ahead and excludes start character
        if k > 0:
            target_data[i, k-1, char_to_idx[ch]] = 1

### Sentence autocompletion using Encoder-Decoder

In [22]:

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from keras.models import load_model
from keras.models import Model

In [23]:
# Encoder:

# Create the input layer of the encoder
encoder_input = Input(shape=(None, len(vocabulary)))
# Create LSTM Layer of size 256
encoder_LSTM = LSTM(256, return_state = True)
# Save encoder output, hidden and cell state
encoder_outputs, encoder_h, encoder_c = encoder_LSTM(encoder_input)
# Save encoder states
encoder_states = [encoder_h, encoder_c]


In [24]:
# Create decoder input layer
decoder_input = Input(shape=(None, len(vocabulary)))

# Create LSTM layer of size 256
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)

# Save decoder output
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)

# Create a `Dense` layer with softmax activation
decoder_dense = Dense(len(vocabulary),activation='softmax')

# Save the decoder output
decoder_out = decoder_dense(decoder_out)

In [25]:
# Combine the encoder and the decoder

# Build model
model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_out])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Print model summary
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 38)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 38)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 302080      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  302080      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
        

### Train the LSTM Network (to generate suffixes corresponding to the prefix sentences)

In [26]:
# Fit the model
model.fit(x=[input_data_prefix, input_data_suffix], y=target_data,
          batch_size=64, epochs=1, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 71378 samples, validate on 17845 samples
Epoch 1/1





71378/71378 [==============================] - 2312s 32ms/step - loss: 0.4352 - val_loss: 0.3898


Run 1: Train on 71378 samples, validate on 17845 samples. My first run was super basic. At a small data set ~3.5% of the overall corpus. Only one reviewer. The amount of processing required is immense and also I am pretty sure I am not doing it the most efficient way but again that does not take away the resources required.One epoch at batch size 64 at a loss of 0.87 took a little over 2 hrs. Small instances mean nothing to text data. (Even for th emost basic run I had to use a AWS c5_4xlarge instance. What is interesting is that loss ratio started at 0.72 and now at 0.52 with past halfway mark. Finished at 2312s 32ms/step - loss:04352 - val loss: 0.3

Run 2: batch size 128 to Epoch 20 (Why 20, from research I see that 20 is a good threshold for smaller data sets such as this one. 

Run 3: batch size 128 to Epoch 500 - reason is to lower loss value.

### Building Inference Models

In [27]:
# Create encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Create decoder input states for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

In [28]:
# Get decoder output and feed it to the dense layer for final output prediction
decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, initial_state=decoder_input_states)
decoder_states = [decoder_h , decoder_c]
decoder_out = decoder_dense(decoder_out)

# Create decoder inference model
decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states, outputs=[decoder_out] + decoder_states)

### Predict the first character using inference models

In [31]:
# Pass input prefix to the Encoder inference model and get the states
inp_seq = input_data_prefix[4:5]
states_val = encoder_model_inf.predict(inp_seq)

# Seed the first character and get output from the decoder 
target_seq = np.zeros((1, 1, len(vocabulary)))
target_seq[0, 0, char_to_idx['\t']] = 1  
# get output from the decoder inference model
decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)

# Find out the most probable next character from the Decoder output
max_val_index = np.argmax(decoder_out[0,-1,:])
sampled_suffix_char = idx_to_char[max_val_index]

In [32]:
# Print the first character
print(sampled_suffix_char)


# Super hilarious, first time I run this my first char returned as space. Imagine the confusion

### Predict the second Character
This is possible by appendding the first character to the existing input to generate the second character

In [41]:
# Insert the generated character from last time to the target sequence 
target_seq = np.zeros((1, 1, len(vocabulary)))
target_seq[0, 0, max_val_index] = 1

# Initialize the decoder state to the states from last iteration
states_val = [decoder_h, decoder_c]

# Get decoder output
decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)

# Get most probable next character and print it.
max_val_index = np.argmax(decoder_out[0,-1,:])
sampled_suffix_char = idx_to_char[max_val_index]
print(sampled_suffix_char)

n


### Auto generate entire sentences
I can give the starting letter and get first and second letter, all that needs to be done is to repeat this process. until I get the end tokem to generate the full suffix. exact same as wine variety but sequence by sequence.

In [42]:
def generate_suffix_sentence(inp_seq):

    # Initialize states value to the final states of the encoder
    states_val = encoder_model_inf.predict(inp_seq)

    # Initialize the target sequence to contain the start token
    target_seq = np.zeros((1, 1, len(vocabulary)))
    target_seq[0, 0, char_to_idx['\t']] = 1

    # Define a variable to store the suffix sentence
    suffix_sent = ''

    # Define stop condition flag
    stop_condition = False

    # Iterate until the end token is found or maximum length of the suffix sentence is reached
    while not stop_condition:

        # Get output from decoder inference model
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)

        # Get most probable next character
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_output_char = idx_to_char[max_val_index]

        # Append the generated char to the suffix sentence
        suffix_sent += sampled_output_char

        # Check if end token is encountered or maximum length of the suffix sentence is exceeded
        if ((sampled_output_char == '\n') or (len(suffix_sent) > max_len_suffix_sent)) :
            stop_condition = True

        # Add the new generated char to the existing target sequence
        target_seq = np.zeros((1, 1, len(vocabulary)))
        target_seq[0, 0, max_val_index] = 1

        # Save state values to use in the next iteration
        states_val = [decoder_h, decoder_c]

    # Return the suffix sentence
    return suffix_sent


In [45]:
# Generate 10 suffixes
for seq_index in range(40):
  
    # Get the next tokenized sentence
    inp_seq = input_data_prefix[seq_index:seq_index+1]
    
    # Generate the suffix sentence
    suffix_sent = generate_suffix_sentence(inp_seq)
    
    # Print the prefix sentence
    print('Prefix Sentence:', prefix_sentences[seq_index])
    
    # Print the suffix sentence
    print('Suffix Sentence:', suffix_sent)

Prefix Sentence: t
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: th
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: thi
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: this
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: this 
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: this b
Suffix Sentence:  and red raspberry in the mouth there are flavors of black cherry and black plum the finish is pleasantly acidic finish

Prefix Sentence: this bl
Suffix Sentence:  

Final thoughts: Even though I threw quite a bit processing power i was able to only train for about 3.5% of my overall data set. It should be possible to increase accuracy of the model by increasing model complexity, train for more epochs, train with a larger data set